In [1]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pathlib import Path

# 1. CARGA DE DATOS
# -------------------------------------------------------------------------
DB_PATH = Path("../data/processed/enigh_unificada.db")
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM tabla_analitica_final", conn)
conn.close()

# 2. LIMPIEZA Y PREPARACIÓN DE VARIABLES (Tu sección 'gen')
# -------------------------------------------------------------------------
# Binarias
df['asis_esc_bin'] = (df['asis_esc'] == 1).astype(int)
df['hombre'] = (df['sexo'] == 1).astype(int)
df['discapacidad'] = df['presenta_alguna_discapacidad'].fillna(0).astype(int) # Asumimos 0 si es nulo
df['indigena'] = (df['etnia'] == 1).astype(int)
df['urbano'] = (df['ambito'] == 'Urbano').astype(int)
df['internet'] = df['tiene_internet'].astype(int)

# Numéricas
df['ingreso_pc_miles'] = df['ingreso_total_hogar_per_capita_sin_jovenes'] / 1000
# Logaritmo (sumamos 0.01 para evitar log(0))
df['ln_ingreso'] = np.log(df['ingreso_pc_miles'] + 0.01) 

df['horas_no_estudio'] = df['total_minutos'] / 60
df['tot_integ_num'] = df['tot_integ'] # Ya viene numérico desde SQL

# Escolaridad Jefe (Recodificación 0-4)
# En SQL ya lo hicimos como 'escolaridad_num' pero era del individuo. 
# Necesitamos la del padre (edu_padre) o madre (edu_madre) o jefe (educa_jefe).
# En tu código Stata usas 'escolaridad_final' derivada de 'escolaridad_101'.
# En la tabla final tenemos 'edu_padre' y 'edu_madre'.
# Usaremos el máximo de los padres como proxy del capital cultural
df['escolaridad_padres'] = df[['edu_padre', 'edu_madre']].max(axis=1).fillna(0)

# Regiones a Dummies (Categorical)
# Statsmodels maneja automáticamente categorías si definimos la columna
df['region'] = df['region'].astype('category')

# 3. FILTRADO DE MUESTRA (Tu lógica de Percentiles)
# -------------------------------------------------------------------------
def get_sample_mask(data, year, min_age, max_age):
    # Filtramos por año y edad primero
    subset = data[(data['anio'] == year) & (data['edad'] >= min_age) & (data['edad'] <= max_age)]
    
    # Calculamos percentiles en ese subgrupo
    p1_ing = subset['ingreso_total_hogar_per_capita_sin_jovenes'].quantile(0.01)
    p99_ing = subset['ingreso_total_hogar_per_capita_sin_jovenes'].quantile(0.99)
    
    p1_hrs = subset['total_minutos'].quantile(0.01)
    p99_hrs = subset['total_minutos'].quantile(0.99)
    
    # Creamos la máscara booleana global
    mask = (
        (data['anio'] == year) &
        (data['edad'] >= min_age) & 
        (data['edad'] <= max_age) &
        (data['ingreso_total_hogar_per_capita_sin_jovenes'] > p1_ing) &
        (data['ingreso_total_hogar_per_capita_sin_jovenes'] < p99_ing) &
        (data['total_minutos'] > p1_hrs) &
        (data['total_minutos'] < p99_hrs) &
        # Filtro de Hijos/Nietos (Parentesco 3xx)
        (data['parentesco'].isin([301, 302, 303, 304]))
    )
    return mask

# Definimos las muestras
mask_m1 = get_sample_mask(df, 2018, 15, 17) # Media Superior 2018
mask_m2 = get_sample_mask(df, 2018, 18, 24) # Superior 2018
mask_m3 = get_sample_mask(df, 2024, 15, 17) # Media Superior 2024
mask_m4 = get_sample_mask(df, 2024, 18, 24) # Superior 2024

print(f"Muestras definidas:\nM1: {mask_m1.sum()}\nM2: {mask_m2.sum()}\nM3: {mask_m3.sum()}\nM4: {mask_m4.sum()}")

# 4. DEFINICIÓN DEL MODELO PROBIT
# -------------------------------------------------------------------------
# Fórmula estilo R/Stata
formula_base = """
asis_esc_bin ~ discapacidad + horas_no_estudio + ln_ingreso + 
               C(escolaridad_padres) + internet + hombre + edad + 
               indigena + urbano + tot_integ_num + 
               C(region) + razon_alumnos_escuelas
"""

# Agregamos las variables específicas por nivel educativo
formula_ms = formula_base + " + ratio_ems_ent + pct_prepa_o_mas"
formula_sup = formula_base + " + ratio_superior_ent + pct_lic_o_mas"

def run_probit(data, mask, formula, model_name):
    print(f"\n--- Estimando {model_name} ---")
    
    # Subconjunto de datos
    df_model = data[mask].copy()
    
    # Eliminar NAs en las variables del modelo
    # Statsmodels falla si hay NAs
    df_model = df_model.dropna(subset=[
        'asis_esc_bin', 'discapacidad', 'horas_no_estudio', 'ln_ingreso',
        'escolaridad_padres', 'internet', 'hombre', 'edad', 'indigena',
        'urbano', 'tot_integ_num', 'region', 'razon_alumnos_escuelas',
        'factor', 'upm'
    ])
    
    # Modelo GLM con familia Binomial y enlace Probit (Equivalente a probit)
    # Usamos freq_weights para el factor de expansión
    model = smf.glm(
        formula=formula, 
        data=df_model, 
        family=sm.families.Binomial(link=sm.families.links.probit()),
        freq_weights=df_model['factor']
    )
    
    # Ajuste con errores robustos agrupados (Cluster por UPM)
    # Esto simula el svy: vce(linearized)
    result = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['upm']})
    
    print(result.summary())
    
    # Efectos Marginales (dydx)
    print(f"Calculando Efectos Marginales para {model_name}...")
    margeff = result.get_margeff(at='overall')
    print(margeff.summary())
    
    return result, margeff

# 5. EJECUCIÓN DE MODELOS
# -------------------------------------------------------------------------

# --- 2018 ---
res_m1, me_m1 = run_probit(df, mask_m1, formula_ms, "M1: Media Sup 2018")
res_m2, me_m2 = run_probit(df, mask_m2, formula_sup, "M2: Superior 2018")

# --- 2024 ---
res_m3, me_m3 = run_probit(df, mask_m3, formula_ms, "M3: Media Sup 2024")
res_m4, me_m4 = run_probit(df, mask_m4, formula_sup, "M4: Superior 2024")

ModuleNotFoundError: No module named 'statsmodels'